In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

import torch
import torch.nn as nn
import torch.functional as F

from options import Options
from models import *
from dataset import *
from utils import *

device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

In [ ]:
print(os.listdir('models'))
# Change this
model_folder = f"models/BERTCombinedModel_finetune/"

In [ ]:
opt = Options()
opt.load_options(model_folder)

state = torch.load(f"{model_folder}/latest.pth", map_location=torch.device('cpu'))
history = torch.load(f"{model_folder}/history.pth")

Dataset = Datasets[opt.dataset]
model = Models[opt.model](opt).to(device)

test_df = pd.read_csv(f"{opt.data_folder}/test.csv")
test_set = Dataset(test_df, opt)
test_loader = DataLoader(test_set, batch_size=30, shuffle=True)

model.load_state_dict(state['state_dict'],)

In [ ]:
# model = model.to(device)
fig, axs = plt.subplots(len(history),1,figsize=(10,10))
test_best = state['epoch']

for i, key in enumerate(history.keys()):
    ax = axs[i]
    ax.set_title(key)

    ax.plot(history[key])
    ax.axvline(test_best, color='red')

In [ ]:
# Confusion matrix and accuracies
y_pred, y_test, test_accuracy = predict(model, test_loader)
print(test_accuracy)

In [ ]:
y_pred, y_test = torch.tensor(y_pred).detach().numpy(), torch.tensor(y_test).detach().numpy()

In [ ]:
# Succ
test_df[y_pred==y_test]

In [ ]:
# Fails
test_df[y_pred!=y_test]

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.xticks(rotation = 45)
plt.title(f"{model_folder.split('/')[1]}")
print(classification_report(y_test, y_pred))